In [ ]:
! python cloud_sql_to_file.py --host 172.16.32.3 --port 1433 --database shapiro --username sqlserver --password 5HjmiJNj4hcnuEyw --table MRRECVH --output MRRECVH

In [ ]:
%pip install -r ../../requirements.txt

In [ ]:
! /root/apache-beam-2.25.0/bin/python -m pip install --upgrade pip

In [ ]:
%pip install apache-beam[gcp]

In [ ]:
%pip install oauth2client==3.0.0

In [ ]:
%pip install httplib2==0.9.2

In [ ]:
import sys
sys.path.insert(0, '/home/jupyter/shapiro-johannes/pysql-beam/pysql-beam')
sys.path.insert(0, '/home/jupyter/shapiro-johannes/pysql-beam')
import logging
import apache_beam as beam
from pysql_beam.sql_io.sql import SQLSource, SQLWriter, ReadFromSQL
from pysql_beam.sql_io.wrapper import MSSQLWrapper
from apache_beam.options.pipeline_options import PipelineOptions, StandardOptions, SetupOptions, GoogleCloudOptions
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ib
from apache_beam.io.gcp.internal.clients.bigquery import TableReference


In [ ]:
dir(TableReference)

In [ ]:
def log(row, level="debug"):
    getattr(logging, level.lower())(row)
    return row


In [ ]:
options = PipelineOptions(host = , port =1433, username='sqlserver', database='shapiro',  password = '5HjmiJNj4hcnuEyw', query = 'SELECT count(*) from ADDIV;')

In [ ]:
options.view_as(StandardOptions).streaming = True

In [ ]:
#options = pipeline_options.view_as(SetupOptions)

In [ ]:
print(options)

In [ ]:
! ls ../..

In [ ]:
options.requirements_file = "../../requirements.txt"

In [ ]:
options.requirements_file

In [ ]:
p = beam.Pipeline(InteractiveRunner(), options=options)

In [ ]:
mysql_data = p | ReadFromSQL(host='172.16.32.3', port=1433,
                                        username='sqlserver', password='5HjmiJNj4hcnuEyw',
                                        databae='shapiro', query='SELECT count(*) from ADDIV;',
                                        wrapper=MSSQLWrapper,
                                        #wrapper=MySQLWrapper,
                                        # wrapper=PostgresWrapper
                                        #
                                        )

In [ ]:
windowed_data = (mysql_data | "window" >> beam.WindowInto(beam.window.FixedWindows(10)))

In [ ]:
! echo $(cat mrrecvh_mrrecvd_ppprice_lj.sql) 

In [14]:
! python cloud_sql_to_file.py  --host '172.16.32.3'  --port 1433  --database 'shapiro'  --username 'sqlserver'  --password '5HjmiJNj4hcnuEyw'  --query "SELECT DATEFROMPARTS(YEAR(MRRECVH.RECV_DATE),MONTH(MRRECVH.RECV_DATE), DAY(MRRECVH.RECV_DATE)) as 'Date', MRRECVH.RECV_DATE, MRRECVD.CONTROL, SUBSTRING(MRRECVD.CONTROL, 9, 2) as 'ADJ', MRRECVD.LINE_TYPE, MRRECVD.SEQUENCE, MRRECVD.ACCOUNT, MRRECVD.COMMODITY, MRRECVD.YARD, MRRECVD.DESCRIPTION, MRRECVD.SETL_NET, MRRECVD.SETL_PRICE, MRRECVD.SETL_UM, MRRECVD.SETL_AMOUNT, PPPrice.EFFECTIVE_DATE, PPPrice.EXPIRE_DATE, PPPrice.DESCRIPTION, PPPrice.MARKET, PPPrice.MARKET_COMM, PPPrice.MARKET_BASE, PPPrice.DIFF_AMOUNT1, PPPrice.DIFF_PERCENT, PPPrice.DIFF_AMOUNT2, PPPrice.PRICE, PPPrice.PRICE_UM, PPPrice.COMMENT, PPMKTD.EFFECTIVE_DATE, PPMKTD.LOW_PRICE, PPMKTD.HIGH_PRICE, PPMKTD.PRICE_UM, PPMKTD.DESCRIPTION FROM MRRECVD INNER JOIN MRRECVH on MRRECVD.COMPANY = MRRECVH.COMPANY and MRRECVD.DIVISION = MRRECVH.DIVISION and MRRECVD.CONTROL = MRRECVH.CONTROL left outer join PPPrice on MRRECVD.COMPANY = PPPrice.COMPANY and MRRECVD.DIVISION = PPPrice.DIVISION and MRRECVD.ACCOUNT = PPPrice.ACCOUNT and MRRECVD.COMMODITY = PPPrice.COMMODITY and MRRECVD.YARD = PPPrice.YARD and MRRECVH.RECV_DATE between PPPrice.EFFECTIVE_DATE AND COALESCE(PPPrice.EXPIRE_DATE, '9999-12-31') left outer join PPMKTD on MRRECVD.COMPANY = PPMKTD.COMPANY and MRRECVD.DIVISION = PPMKTD.DIVISION and PPPrice.MARKET_COMM = PPMKTD.COMMODITY and PPPrice.MARKET = PPMKTD.MARKET and DATEFROMPARTS(YEAR(MRRECVH.RECV_DATE),MONTH(MRRECVH.RECV_DATE),1) = PPMKTD.EFFECTIVE_DATE where MRRECVH.RECV_DATE > '2019-01-01'"  --output_table 'shapiro-metals:shapiro_analytics.mrrecvh_mrrecvd_ppprice_lj' --temp_location "gs://shapiro-dataflow-staging/tmp" --experiment=use_beam_bq_sink #--region us-central1 --runner DataflowRunner  --project shapiro-metals  


from inside the readFromSQL class, source: <class 'pysql_beam.sql_io.wrapper.MSSQLWrapper'>
pagination query do fn wrapper: <class 'pysql_beam.sql_io.wrapper.MSSQLWrapper'>
/opt/conda/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery.py:1677: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  experiments = p.options.view_as(DebugOptions).experiments or []
/opt/conda/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery_file_loads.py:900: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = p.options.view_as(GoogleCloudOptions).temp_location
host 172.16.32.3
port 1433
database shapiro
username sqlserver
password 5HjmiJNj4hcnuEyw
query SELECT DATEFROMPARTS(YEAR(MRRECVH.RECV_DATE),MONTH(MRRECVH.RECV_DATE), DAY(MRRECVH.RECV_DATE)) as 'Date', MRRECVH.RECV_DATE, MRRECVD.CONTROL, SUBSTRING(MRRECVD.CONTROL, 9, 2) a

In [15]:
! python cloud_sql_to_file.py  --host '172.16.32.3'  --port 1433  --database 'shapiro'  --username 'sqlserver'  --password '5HjmiJNj4hcnuEyw'  --query "SELECT COMPANY, DIVISION, MARKET, COMMODITY, EFFECTIVE_DATE, ISSUE_DATE, DESCRIPTION, LOW_PRICE, HIGH_PRICE, PRICE_UM, CREATEDATE, CREATETIME, CREATEUSER, CREATESTATION, LASTDATE, LASTTIME, LASTUSER, LASTSTATION FROM shapiro.dbo.PPMKTD where EFFECTIVE_DATE >= '2019-01-01';"  --output_table 'shapiro-metals:shapiro_analytics.ppmktd_filter_effective_date' --temp_location "gs://shapiro-dataflow-staging/tmp" --experiment=use_beam_bq_sink #--region us-central1 --runner DataflowRunner  --project shapiro-metals  


from inside the readFromSQL class, source: <class 'pysql_beam.sql_io.wrapper.MSSQLWrapper'>
pagination query do fn wrapper: <class 'pysql_beam.sql_io.wrapper.MSSQLWrapper'>
/opt/conda/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery.py:1677: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  experiments = p.options.view_as(DebugOptions).experiments or []
/opt/conda/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery_file_loads.py:900: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = p.options.view_as(GoogleCloudOptions).temp_location
host 172.16.32.3
port 1433
database shapiro
username sqlserver
password 5HjmiJNj4hcnuEyw
query SELECT COMPANY, DIVISION, MARKET, COMMODITY, EFFECTIVE_DATE, ISSUE_DATE, DESCRIPTION, LOW_PRICE, HIGH_PRICE, PRICE_UM, CREATEDATE, CREATETIME, CREATEUSER, CREATESTATION, LASTDA

In [3]:
import pyodbc
server = '172.16.32.3'
database = 'shapiro'
username = 'sqlserver'
password = '5HjmiJNj4hcnuEyw'
driver='{ODBC Driver 17 for SQL Server}'
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1443;DATABASE='+database+';UID='+username+';PWD='+password)
cursor = cnxn.cursor()
cursor.execute('SELECT * INTO TempTable FROM DITRIP ALTER TABLE TempTable DROP COLUMN SIGNATURE ');
cursor.close()
cnxn.close()

In [4]:
! python cloud_sql_to_file.py  --host '172.16.32.3'  --port 1433  --database 'shapiro'  --username 'sqlserver'  --password '5HjmiJNj4hcnuEyw'  --query "SELECT * FROM TempTable"  --output_table 'shapiro-metals:shapiro_analytics.DITRIP' --temp_location "gs://shapiro-dataflow-staging/tmp" --experiment=use_beam_bq_sink #--region us-central1 --runner DataflowRunner  --project shapiro-metals  


from inside the readFromSQL class, source: <class 'pysql_beam.sql_io.wrapper.MSSQLWrapper'>
pagination query do fn wrapper: <class 'pysql_beam.sql_io.wrapper.MSSQLWrapper'>
/opt/conda/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery.py:1677: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  experiments = p.options.view_as(DebugOptions).experiments or []
/opt/conda/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery_file_loads.py:900: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = p.options.view_as(GoogleCloudOptions).temp_location
host 172.16.32.3
port 1433
database shapiro
username sqlserver
password 5HjmiJNj4hcnuEyw
query SELECT * FROM TempTable
we're in the process method now; source.client: <pysql_beam.sql_io.wrapper.MSSQLWrapper object at 0x7f685061f210>
select count(1) as row_count from (SEL

Correctly done: `GLTRAN`, `AMTRAND` `CITRAND` `CITRANH` `DIRTRIP` `GLCHARH` `MSSHIPD` `MSSHIPH` `SIINVD` `SMDITRP` `DITRIP` `APADV` `SRSHIP` `SIINVH` `CITRAND` `DIIMAGE` `DIRTRIP` `MSSHIPD` `CACAM` `SMPSTERR` `MRIMAGE` `FRPLAND` `FRPLANH` `MRVIN` `SMCRITERIA` `MRINSP` `MSIMAGE`
  

In [ ]:
print(1)

In [ ]:
DITRIP